# GPU Memory and CUDA.jl
This notebook contains notes on the GPU memory layout, processing, and some of the lower-level access to the CUDA API which interfaces with those aspects, as well as the specific CUDA.jl bindings.

I watched a series of videos last night, which explain how CUDA functions
- [CUDA Tutorials](https://www.youtube.com/watch?v=m0nhePeHwFs&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK) by Creel
- [Intro to Cuda](https://www.youtube.com/watch?v=cRY5utouJzQ) by Josh Holloway

## Summary
In short, the GPU memory and architecture looks something like this:

![](./gpumem.png)

### Computation
The principle units of GPU processing are *Threads*, which in CUDA are arranged into *Blocks*. GPUs use this block-based processing design so that the same code can run irrespective of the GPU model itself, with more powerful GPUs churning through more blocks at a time.

The GPU has a number of *Streaming Multiprocessors* (SM), each consisting of a number of *Streaming Processors* (SP), which are scalar lanes, running a single thread at a time. A SM schedules instructions to the SP as it executes a block, through the *warp scheduler*.

A block is exectuted in *warps*, where a single *warp* is usually defined as the execution of 32 simultaneous threads (related to memory access transaction specifics, see [this SO answer](https://stackoverflow.com/a/11821971)).

A single block can hold up to 1024 threads, and there is no practical limit on the number of blocks you can define. 

An overall scheduler on the GPU will distribute blocks to the SMs, which in turn will execute a number of warps per clock cycle until all of the blocks have been completed.

In CUDA, when we launch a processing kernel (a routine/algorithm/function) on the GPU, we can specifiy the block layout, and within those blocks, the number of threads.

The block layout is known as a *grid*.

There's elements of tradeoffs between number of threads and the shared memory in a block, which is why you may want to limit these things, but I haven't investigated consequences of not just maxing the number of threads per block beyond that. Something to learn in the future.

### Memory
Now this is the bit particularly relevant to my initial investigation yesterday.

Each block has shared memory, which is stored on-chip of the SM. This memory is typically rather small (10s of kilobytes), and is used for intercommunication between threads. The RW operations are consequently very fast by proximity. Shared memory is accessed through an L1 cache, so write operations to shared memory must be synchronised.

Each thread has register memory (unlike CPUs, there are many thousand GPU registers) for storing immediate data, and any spillover from register memory is stored in the thread's local memory. Now despite the name, the local memory is off-chip, stored in the GPUs equivalent of DRAM, and so RW operations are slow. The local memory is owned and exclussively accessible by the running thread, as, of course, as the registers.

Then there is the global memory, which is host not only to each thread's local memory, but itself represents memory that each thread can read and write to, albeit slow by distance, and not write-threadsafe.

The global memory also houses the constant memory (sections of read-only written by the CPU and locked when a kernel launches), and the texture memory, which is special interpolated read-only memory.

On modern GPUs, the global memory is accessed through an L2 cache.


## Learning about a device
The CUDA library provides different tools for obtaining information on GPU devices, such as the command line `nvidia-smi` for a whole range of information. However CUDA.jl provides bindings for much of what we would need:

In [4]:
using CUDA

In [7]:
dev = collect(CUDA.devices())[1]

CuDevice(0): GeForce GTX 980

The [compute capability](https://forums.developer.nvidia.com/t/compute-capability/110091):

In [8]:
CUDA.capability(dev)

v"5.2.0"

Numbers of threas per warp:

In [9]:
CUDA.warpsize(dev)

32

Number of threads per block:

In [13]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

1024

The amount of shared memory per block:

In [15]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MAX_SHARED_MEMORY_PER_BLOCK)

49152

which is about 46 KB.

Total constant memory:

In [16]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_TOTAL_CONSTANT_MEMORY)

65536

Maximum number of registers per block:

In [17]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MAX_REGISTERS_PER_BLOCK)

65536

In [19]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_REGISTERS_PER_BLOCK)

65536

Multiprocessor count:

In [20]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT)

16

Something I was toying with, which is an option again now that I understand the memory layout better, was storing my calculated geodesics in texture memory so I can float index them. Texture memory can be stored in 1, 2 or 3 dimensions -- and we can see the maximum amount of memory in each:

In [21]:
CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MAXIMUM_TEXTURE1D_WIDTH)

65536

## Re-evaluating my approach
Yesterday I focused on trying to use the `map` command defined for `CuArray`, which invokes its own kernel function. As such, I don't think I have fine tuned control over the memory (although I could be wrong), and instead may have to write my own kernel to achieve this level of geodesic tracing.

I want to try a few more things with `CuArray`s first before I give up and learn about writing Kernels: